## Synthetic Data Creation

In order to create the data to analyse, a synthetic dataset with 5000 customer records has been created.
Based on the customers dataset the following datasets are created, in order to enable a simulation of data analysis.

* subscription_events
* transactions
* marketing_campaigns
* engagement_logs

### Setting up the environment with necessary modules and libraries

In [110]:
%pip install pandas
%pip install faker

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\Family Bod\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\Family Bod\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [111]:
from datetime import timedelta, date
import random
import csv
from os import path, mkdir

import pandas as pd
from faker import Faker
import numpy as np

### Data generation

In [112]:
faker = Faker()
Faker.seed(42)
random.seed(42)

In [113]:
n_customers = 5000
signup_date = faker.date_between(start_date='-8y', end_date='-7y')

customers = []
for i in range(n_customers):    
    customers.append({
        'customer_id': i + 1,
        'signup_date': signup_date,
        'age': random.randint(18, 65),
        'gender': random.choice(['Male', 'Female', 'Other']),
        'region': random.choice(['DE', 'BE', 'FR', 'NL', 'AT', 'DK']),
        'device_type': random.choice(['iOS', 'Android', 'Web']),
        'acquisition_channel': random.choice(['Ads', 'Organic', 'Referral', 'Partner', 'Other'])
    })

    signup_date = faker.date_between(start_date=signup_date, end_date=signup_date+timedelta(days=2))

customers_df = pd.DataFrame(customers)

In [114]:
customers_df

,customer_id,signup_date,age,gender,region,device_type,acquisition_channel
0,1,2017-12-28,58,Male,DE,Web,Referral
1,2,2017-12-28,33,Male,BE,Web,Ads
2,3,2017-12-28,61,Other,AT,iOS,Other
3,4,2017-12-28,45,Male,DE,iOS,Organic
4,5,2017-12-28,32,Other,AT,iOS,Other
...,...,...,...,...,...,...,...
4995,4996,2024-12-06,44,Female,NL,iOS,Partner
4996,4997,2024-12-07,32,Other,NL,iOS,Ads
4997,4998,2024-12-07,54,Other,NL,Android,Referral
4998,4999,2024-12-07,21,Female,AT,Android,Referral


In [115]:
plan_fees = {'Free': 0, 'Basic': 9.99, 'Premium': 19.99}

subscription_events = []
for i in range(n_customers):
    start_date = customers_df.loc[i, 'signup_date']
    max_end_date = date.today()
    churned = random.choice([True, False])
    end_date = faker.date_between(start_date, max_end_date) if churned else None

    subscription_events.append(
        {
            'customer_id': customers_df.loc[i, 'customer_id'],
            'start_date': start_date,
            'end_date': end_date,
            'plan_type': random.choice(list(plan_fees.keys())),
            'is_active': 0 if churned else 1,
            'churn_flag': 1 if churned else 0,
            'churn_reason': random.choice(['Don\'t need it anymore', 'Found better product', 'Not specified']) if churned else None
        },
    )

subscription_events_df = pd.DataFrame(subscription_events)

In [116]:
subscription_events_df

,customer_id,start_date,end_date,plan_type,is_active,churn_flag,churn_reason
0,1,2017-12-28,None,Free,1,0,None
1,2,2017-12-28,None,Premium,1,0,None
2,3,2017-12-28,2023-03-28,Premium,0,1,Don't need it anymore
3,4,2017-12-28,None,Basic,1,0,None
4,5,2017-12-28,2025-02-02,Basic,0,1,Not specified
...,...,...,...,...,...,...,...
4995,4996,2024-12-06,2025-02-15,Basic,0,1,Don't need it anymore
4996,4997,2024-12-07,None,Basic,1,0,None
4997,4998,2024-12-07,2025-03-24,Basic,0,1,Don't need it anymore
4998,4999,2024-12-07,None,Basic,1,0,None


In [117]:
transactions = []

for i, row in subscription_events_df.iterrows():
    if row['plan_type'] == 'Free':
        continue

    start = row['start_date']
    end = row['end_date'] or date.today()
    months = (end.year - start.year) * 12 + (end.month - start.month)

    for m in range(months):
        transaction_date = start + timedelta(days= m * 30)
        transactions.append({
            'customer_id': row['customer_id'],
            'transaction_date': transaction_date,
            'amount_in_euro': plan_fees[row['plan_type']],
            'payment_method': random.choice(['Credit Card', 'PayPal', 'Bank Transfer'])
        })

transactions_df = pd.DataFrame(transactions)
transactions_df = transactions_df.sort_values(by=['transaction_date', 'customer_id'], ascending=True).reset_index(drop=True)
transactions_df.insert(loc=0, column='transaction_id', value=range(1, len(transactions_df) + 1), allow_duplicates=False)

In [118]:
transactions_df

,transaction_id,customer_id,transaction_date,amount_in_euro,payment_method
0,1,2,2017-12-28,19.99,Credit Card
1,2,3,2017-12-28,19.99,Credit Card
2,3,4,2017-12-28,9.99,PayPal
3,4,5,2017-12-28,9.99,Credit Card
4,5,9,2017-12-29,9.99,Bank Transfer
...,...,...,...,...,...
112650,112651,4864,2025-03-29,19.99,Credit Card
112651,112652,4865,2025-03-29,9.99,Credit Card
112652,112653,4925,2025-03-30,9.99,Credit Card
112653,112654,4927,2025-03-30,9.99,Credit Card


In [119]:
campaigns = []
for i in range(n_customers):
    for _ in range(random.randint(1, 3)):
        campaigns.append({
            'campaign_id': faker.uuid4(),
            'customer_id': customers_df.loc[i, 'customer_id'],
            'channel': random.choice(['Email', 'Push', 'SMS']),
            'open_rate': round(random.uniform(0.1, 0.9), 2),
            'click_through_rate': round(random.uniform(0.05, 0.5), 2),
            'conversion': random.choice([0, 1])
        })

campaigns_df = pd.DataFrame(campaigns)

In [120]:
campaigns_df

,campaign_id,customer_id,channel,open_rate,click_through_rate,conversion
0,94fd0594-ecf9-4245-be5f-7e23704c19e2,1,Email,0.84,0.24,1
1,ca826730-1997-4e2e-a224-98130a6b3e3d,2,Push,0.79,0.43,0
2,604a1fe2-9cc8-4772-a1d7-8199d8ed0649,2,SMS,0.26,0.29,0
3,e12f3b21-0400-4940-a862-8b4ec714d549,3,SMS,0.80,0.22,0
4,01d96e96-01a0-4047-94aa-3ca37ca9e68f,3,Email,0.69,0.22,1
...,...,...,...,...,...,...
10048,0c389a24-a9fb-4275-8d0b-44e9fe7df350,4998,SMS,0.48,0.06,1
10049,55412f59-f322-486c-9f81-78d21c1b2fdd,4999,Push,0.33,0.46,0
10050,7471527a-765c-4768-8094-2f81a4b912f1,5000,Email,0.86,0.15,0
10051,49a917d6-c439-43ed-ba8d-24d6dbdadb93,5000,Email,0.80,0.10,1


In [121]:
engagement_logs = []

actions = ['view_balance', 'make_payment', 'transfer_funds', 'view_offers', 'update_profile']

for i, row in subscription_events_df.iterrows():
    num_logs = random.randint(5, 30)
    for _ in range(num_logs):
        login_time = faker.date_time_between(start_date=row['start_date'], end_date=row['end_date'] if row['end_date'] != None else 'now')
        engagement_logs.append({
            'customer_id': subscription_events_df.loc[i, 'customer_id'],
            'login_time': login_time,
            'action_type': random.choice(actions),
            'session_duration_in_minutes': round(random.uniform(1, 60), 2)
        })

engagement_df = pd.DataFrame(engagement_logs)
engagement_df = engagement_df.sort_values(by='login_time', ascending=True).reset_index(drop=True)


In [122]:
engagement_df

,customer_id,login_time,action_type,session_duration_in_minutes
0,4,2018-01-02 20:15:15,update_profile,12.50
1,21,2018-01-03 07:32:49,make_payment,15.01
2,21,2018-01-05 05:12:40,view_offers,29.41
3,13,2018-01-08 13:07:38,make_payment,32.40
4,37,2018-01-12 00:01:54,view_offers,39.70
...,...,...,...,...
87024,1413,2025-04-24 12:10:02,view_balance,29.03
87025,4725,2025-04-24 12:42:32,update_profile,5.18
87026,4113,2025-04-24 13:07:05,view_balance,47.21
87027,4540,2025-04-24 13:07:55,transfer_funds,51.50


### Saving the synthesized data locally in .csv format

In [123]:
file_path = path.curdir + '/csv_files/'

if not path.dirname('file_path') in path.curdir:
    mkdir(file_path)
else:
    pass

customers_df.to_csv(path_or_buf=file_path + 'customers.csv')
subscription_events_df.to_csv(path_or_buf=file_path + 'subscription_events.csv')
transactions_df.to_csv(path_or_buf=file_path + 'transactions.csv')
campaigns_df.to_csv(path_or_buf=file_path + 'marketing_campaigns.csv')
engagement_df.to_csv(path_or_buf=file_path + 'engagement_logs.csv')